# surrealdb

testing [surrealdb.rs](https://crates.io/crates/surrealdb) rust client using [jupyter labs notebooks](https://jupyter.org/try)

In [10]:
// install packages
:dep color-eyre = "0.6.2"
// :dep uuid = { version = "1.3.0", features = ["v4", "macro-diagnostics", "fast-rng", "serde"] }
:dep serde = "1.0.189"
:dep serde_json = "1.0.107"
:dep tokio = { version = "1.33.0", features = ["full"] }
:dep surrealdb = "1"
:dep chrono = "0.4.31"

In [11]:
use serde::{Deserialize, Serialize};
use serde_json::json;
use std::borrow::Cow;
use surrealdb::sql;
use surrealdb::sql::Thing;
use surrealdb::Surreal;
use surrealdb::engine::remote::ws::Ws;
use surrealdb::opt::auth::Root;
use chrono::{DateTime, Utc};

In [15]:

#[derive(Debug, Serialize, Deserialize)]
struct Name {
    firstname: String,
    lastname: String,
}

#[derive(Debug, Serialize, Deserialize)]
pub struct Identity {
    id: Option<Thing>,
    email: String,
    login: String,
    password: String,
    preferred: bool,
    provider: Provider,
}

#[derive(Debug, Serialize, Deserialize)]
pub struct Provider {
    id: Option<Thing>,
    name: String,
}

#[derive(Debug, Serialize, Deserialize)]
struct Person {
    #[serde(skip_serializing)]
    id: Option<Thing>,
    nickname: String,
    name: Name,
    identity: Identity,
    created: DateTime<Utc>, // UTC timestamp for creation time
    updated: DateTime<Utc>, // UTC timestamp for update time
}

#[derive(Debug, Serialize, Deserialize)]
struct NewPerson {
    name: Name,
    nickname: String,
    status: String,
    email: String,
    login: String,
    password: String,
    preferred: bool,
    provider: String,
    created: DateTime<Utc>, // UTC timestamp for creation time
    updated: DateTime<Utc>, // UTC timestamp for update time
}


### request payload

The following data payload will be used to create a record in edgedb and used for retrieval purposes
```json
{
  "name": {
    "firstname": "tony",
    "lastname": "stark",
  },
  "nickname": "ironman",
  "status": "PENDING",
  "email": "ironman@avengers.com",
  "login": "@ironman",
  "password": "$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He",
  "preferred": true,
  "provider": "ardency"
}
```

In [16]:
let db = Surreal::new::<Ws>("localhost:8000").await?;

// Signin as a namespace, database, or root user
db.signin(Root {
    username: "root",
    password: "root",
})
.await?;

// Select a specific namespace and database
db.use_ns("labs").use_db("labs").await?;

// get current time
let current_time = Utc::now(); 

// Create a new person with a random ID
let tobie: Vec<Person> = db
    .create("person")
    .content(Person {
        id: None,
        nickname: "ironman".into(),
        name: Name {
            firstname: "Tony".into(),
            lastname: "Stark".into(),
        },
        identity: Identity {
            id: None,
            login: "@ironman".into(),
            email: "ironman@avengers.com".into(),
            password: "$2b$12$lH23QLU6pRf9g8jJW91HvOrPbGDFzwZ6x8Lz0jKkMQ7Bmgf1Sw9He".into(),
            preferred: true,
            provider: Provider {
                id: None,
                name: "ardency".into(),
            }
        },
        
        created: current_time,
        updated: current_time, 
    })
    .await?;

// Select all people records
let people: Vec<Person> = db.select("person").await?;

println!("{:?}",people)

The type of the variable people was redefined, so was lost.
The type of the variable tobie was redefined, so was lost.
thread 'tokio-runtime-worker' panicked at 'there is no reactor running, must be called from the context of a Tokio 1.x runtime', /Users/pepeye/.cargo/registry/src/index.crates.io-6f17d22bba15001f/tokio-1.33.0/src/time/interval.rs:138:26
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: tokio::runtime::scheduler::Handle::current
   3: tokio::time::sleep::Sleep::new_timeout
   4: tokio::time::interval::interval
   5: surrealdb::api::engine::remote::ws::native::router::{{closure}}
   6: tokio::runtime::task::core::Core<T,S>::poll
   7: tokio::runtime::task::harness::Harness<T,S>::poll
   8: tokio::runtime::scheduler::multi_thread::worker::Context::run_task
   9: tokio::runtime::scheduler::multi_thread::worker::Context::run
  10: tokio::runtime::context::scoped::Scoped<T>::set
  11: tokio::runtime::context::runtime::enter_runtime
  12: tokio: